In [ ]:
    def generate_risk_report(self):
        """
        Generate comprehensive risk report with visualizations
        """
        if not self.performance_metrics or not self.risk_metrics:
            raise ValueError("Please calculate performance and risk metrics first.")
        
        # Create comprehensive report
        report = {
            'Portfolio Summary': self._get_portfolio_summary(),
            'Performance Metrics': self.performance_metrics,
            'Risk Metrics': self.risk_metrics,
            'Scenario Analysis': self.scenario_analysis(),
            'Visualizations': self._create_visualizations()
        }
        
        return report
    
    def _get_portfolio_summary(self):
        """Get portfolio summary information"""
        if self.portfolio_data is None:
            return {}
        
        # Calculate current portfolio value
        total_value = 0
        positions = []
        
        for _, row in self.portfolio_data.iterrows():
            symbol = row['symbol']
            shares = row['shares']
            purchase_price = row['purchase_price']
            
            if symbol in self.price_data:
                current_price = self.price_data[symbol]['Close'].iloc[-1]
                current_value = shares * current_price
                total_value += current_value
                
                positions.append({
                    'Symbol': symbol,
                    'Shares': shares,
                    'Purchase Price': purchase_price,
                    'Current Price': current_price,
                    'Current Value': current_value,
                    'Gain/Loss': current_value - (shares * purchase_price),
                    'Gain/Loss %': (current_price / purchase_price - 1) * 100
                })
        
        return {
            'Total Portfolio Value': total_value,
            'Number of Positions': len(positions),
            'Positions': positions
        }
    
    def _create_visualizations(self):
        """Create risk and performance visualizations"""
        if not self.returns_data:
            return {}
        
        # Create plots
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        
        # 1. Portfolio Performance
        if 'PORTFOLIO' in self.returns_data:
            portfolio_returns = self.returns_data['PORTFOLIO']
            cumulative_returns = (1 + portfolio_returns).cumprod()
            axes[0, 0].plot(cumulative_returns.index, cumulative_returns.values)
            axes[0, 0].set_title('Portfolio Cumulative Returns')
            axes[0, 0].set_ylabel('Cumulative Return')
            axes[0, 0].grid(True)
        
        # 2. Risk-Return Scatter
        if self.performance_metrics:
            symbols = list(self.performance_metrics.keys())
            returns = [self.performance_metrics[s]['Annualized Return'] for s in symbols]
            volatilities = [self.performance_metrics[s]['Volatility'] for s in symbols]
            
            axes[0, 1].scatter(volatilities, returns, alpha=0.7)
            for i, symbol in enumerate(symbols):
                axes[0, 1].annotate(symbol, (volatilities[i], returns[i]))
            axes[0, 1].set_xlabel('Volatility')
            axes[0, 1].set_ylabel('Annualized Return')
            axes[0, 1].set_title('Risk-Return Profile')
            axes[0, 1].grid(True)
        
        # 3. Drawdown Analysis
        if 'PORTFOLIO' in self.returns_data:
            portfolio_returns = self.returns_data['PORTFOLIO']
            cumulative_returns = (1 + portfolio_returns).cumprod()
            running_max = cumulative_returns.expanding().max()
            drawdown = (cumulative_returns - running_max) / running_max
            
            axes[1, 0].fill_between(drawdown.index, drawdown.values, 0, alpha=0.3, color='red')
            axes[1, 0].set_title('Portfolio Drawdown')
            axes[1, 0].set_ylabel('Drawdown')
            axes[1, 0].grid(True)
        
        # 4. Correlation Heatmap
        if len(self.returns_data) > 1:
            returns_df = pd.DataFrame(self.returns_data)
            correlation_matrix = returns_df.corr()
            
            sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, ax=axes[1, 1])
            axes[1, 1].set_title('Returns Correlation Matrix')
        
        plt.tight_layout()
        plt.show()
        
        return "Visualizations created successfully"


In [ ]:
# Example usage and demonstration
def main():
    """
    Main function to demonstrate the portfolio risk analyzer
    """
    print("=== Portfolio Risk Analysis Platform ===")
    print("AI-driven portfolio risk and analytics platform for professional investment managers")
    print()
    
    # Initialize analyzer
    analyzer = PortfolioRiskAnalyzer()
    
    # Load portfolio data from inputs file
    print("Loading portfolio data from inputs...")
    portfolio_data = analyzer.load_portfolio_from_inputs("inputs/portfolio_data.json")
    print(f"Portfolio loaded with {len(portfolio_data)} positions")
    print()
    
    # Display portfolio holdings
    print("Portfolio Holdings:")
    print(portfolio_data.to_string(index=False))
    print()
    
    # Fetch price data
    print("Fetching historical price data...")
    price_data = analyzer.fetch_price_data()
    print()
    
    # Calculate returns
    print("Calculating returns...")
    returns_data = analyzer.calculate_returns()
    print()
    
    # Calculate performance metrics
    print("Calculating performance metrics...")
    performance_metrics = analyzer.calculate_performance_metrics()
    print()
    
    # Calculate risk metrics
    print("Calculating risk metrics...")
    risk_metrics = analyzer.calculate_risk_metrics()
    print()
    
    # Generate comprehensive report
    print("Generating risk report...")
    report = analyzer.generate_risk_report()
    
    # Display key metrics
    print("\n=== PORTFOLIO PERFORMANCE SUMMARY ===")
    if 'PORTFOLIO' in performance_metrics:
        portfolio_metrics = performance_metrics['PORTFOLIO']
        print(f"Total Return: {portfolio_metrics['Total Return']:.2%}")
        print(f"Annualized Return: {portfolio_metrics['Annualized Return']:.2%}")
        print(f"Volatility: {portfolio_metrics['Volatility']:.2%}")
        print(f"Sharpe Ratio: {portfolio_metrics['Sharpe Ratio']:.2f}")
        print(f"Max Drawdown: {portfolio_metrics['Max Drawdown']:.2%}")
        print(f"Upside Capture: {portfolio_metrics['Upside Capture']:.2f}")
        print(f"Downside Capture: {portfolio_metrics['Downside Capture']:.2f}")
    
    print("\n=== RISK METRICS ===")
    if 'PORTFOLIO' in risk_metrics:
        portfolio_risk = risk_metrics['PORTFOLIO']
        print(f"Beta: {portfolio_risk['Beta']:.2f}")
        print(f"VaR 95%: {portfolio_risk['VaR 95%']:.2%}")
        print(f"VaR 99%: {portfolio_risk['VaR 99%']:.2%}")
        print(f"CVaR 95%: {portfolio_risk['CVaR 95%']:.2%}")
    
    print("\n=== SCENARIO ANALYSIS ===")
    scenarios = analyzer.scenario_analysis()
    for scenario, result in scenarios.items():
        print(f"{scenario}: {result}")
    
    return analyzer, report

# Run the analysis
if __name__ == "__main__":
    analyzer, report = main()


In [ ]:
# Load portfolio metadata if available
def load_portfolio_metadata(metadata_path="inputs/portfolio_metadata.json"):
    """
    Load additional portfolio metadata for enhanced analysis
    """
    try:
        import json
        with open(metadata_path, 'r') as f:
            metadata = json.load(f)
        print("Portfolio metadata loaded successfully")
        return metadata
    except FileNotFoundError:
        print("No metadata file found, proceeding with basic analysis")
        return None
    except Exception as e:
        print(f"Error loading metadata: {e}")
        return None

# Load metadata
metadata = load_portfolio_metadata()
if metadata:
    print(f"Portfolio: {metadata['portfolio_info']['name']}")
    print(f"Description: {metadata['portfolio_info']['description']}")
    print(f"Total Initial Investment: ${metadata['portfolio_info']['total_initial_investment']:,.2f}")
    print()


In [ ]:
# Enhanced portfolio analysis with sector breakdown
def analyze_portfolio_composition(portfolio_data, metadata=None):
    """
    Analyze portfolio composition and sector allocation
    """
    print("=== PORTFOLIO COMPOSITION ANALYSIS ===")
    
    # Basic portfolio stats
    total_positions = len(portfolio_data)
    total_investment = sum([row['shares'] * row['purchase_price'] for _, row in portfolio_data.iterrows()])
    
    print(f"Total Positions: {total_positions}")
    print(f"Total Initial Investment: ${total_investment:,.2f}")
    print()
    
    # Sector analysis if available
    if 'sector' in portfolio_data.columns:
        print("Sector Allocation:")
        sector_analysis = portfolio_data.groupby('sector').agg({
            'shares': 'sum',
            'purchase_price': 'mean',
            'symbol': 'count'
        }).round(2)
        sector_analysis.columns = ['Total Shares', 'Avg Price', 'Positions']
        
        # Calculate sector weights
        sector_weights = []
        for sector in sector_analysis.index:
            sector_positions = portfolio_data[portfolio_data['sector'] == sector]
            sector_value = sum([row['shares'] * row['purchase_price'] for _, row in sector_positions.iterrows()])
            weight = sector_value / total_investment
            sector_weights.append(weight)
        
        sector_analysis['Weight'] = [f"{w:.1%}" for w in sector_weights]
        print(sector_analysis)
        print()
    
    # Top holdings by value
    print("Top 5 Holdings by Initial Value:")
    portfolio_data['initial_value'] = portfolio_data['shares'] * portfolio_data['purchase_price']
    top_holdings = portfolio_data.nlargest(5, 'initial_value')[['symbol', 'shares', 'purchase_price', 'initial_value']]
    print(top_holdings.to_string(index=False))
    print()
    
    return sector_analysis if 'sector' in portfolio_data.columns else None

# Run composition analysis
sector_analysis = analyze_portfolio_composition(portfolio_data, metadata)


# Integration Test and Usage Instructions

## How to Use the Portfolio Risk Analysis Platform

### Step 1: Generate Portfolio Data
1. Run the `generate data.ipynb` notebook first
2. This will create the following files in the `inputs/` directory:
   - `portfolio_data.json` - Main portfolio data
   - `portfolio_data.csv` - CSV version for Excel compatibility  
   - `portfolio_metadata.json` - Additional metadata and sector analysis

### Step 2: Run Risk Analysis
1. Run this `report.ipynb` notebook
2. The system will automatically load data from the inputs files
3. Generate comprehensive risk analysis and visualizations

### Data Flow:
```
generate data.ipynb → inputs/portfolio_data.json → report.ipynb → Risk Analysis
```

### Features Available:
- ✅ Portfolio data generation with realistic sample data
- ✅ Sector allocation and composition analysis
- ✅ Performance metrics calculation
- ✅ Risk metrics and VaR analysis
- ✅ Scenario analysis and stress testing
- ✅ Comprehensive visualizations
- ✅ Excel export capability
- ✅ Natural language query interface


In [ ]:
# Test the complete integration
print("=== TESTING INTEGRATION ===")
print("Testing portfolio data loading and analysis pipeline...")
print()

# Test 1: Check if inputs files exist
import os
inputs_files = [
    "inputs/portfolio_data.json",
    "inputs/portfolio_data.csv", 
    "inputs/portfolio_metadata.json"
]

print("Checking for required input files:")
for file in inputs_files:
    if os.path.exists(file):
        print(f"✅ {file} - Found")
    else:
        print(f"❌ {file} - Missing")

print()

# Test 2: Verify portfolio data structure
if os.path.exists("inputs/portfolio_data.json"):
    try:
        import json
        with open("inputs/portfolio_data.json", 'r') as f:
            test_data = json.load(f)
        print(f"✅ Portfolio data structure valid - {len(test_data)} positions")
        
        # Check required fields
        required_fields = ['symbol', 'shares', 'purchase_date', 'purchase_price']
        if all(field in test_data[0] for field in required_fields):
            print("✅ All required fields present")
        else:
            print("❌ Missing required fields")
            
    except Exception as e:
        print(f"❌ Error reading portfolio data: {e}")
else:
    print("❌ Portfolio data file not found - please run generate data.ipynb first")

print()

# Test 3: Integration status
print("=== INTEGRATION STATUS ===")
if all(os.path.exists(f) for f in inputs_files):
    print("✅ All input files present - Ready for risk analysis")
    print("✅ Data generation complete")
    print("✅ Report notebook ready to run")
else:
    print("❌ Missing input files - Please run generate data.ipynb first")
    print("   Then run this report.ipynb notebook")

print()
print("Next steps:")
print("1. If files are missing, run generate data.ipynb")
print("2. Run the main() function below to start risk analysis")
print("3. Or run individual analysis steps as needed")


In [ ]:
# Additional utility functions for advanced analytics

def create_portfolio_data_template():
    """
    Create a template for portfolio data that users can fill in
    """
    template = {
        'portfolio_data': [
            {'symbol': 'AAPL', 'shares': 100, 'purchase_date': '2023-01-15', 'purchase_price': 150.00},
            {'symbol': 'MSFT', 'shares': 50, 'purchase_date': '2023-02-20', 'purchase_price': 250.00},
            {'symbol': 'GOOGL', 'shares': 30, 'purchase_date': '2023-03-10', 'purchase_price': 2800.00},
            {'symbol': 'SPY', 'shares': 200, 'purchase_date': '2023-01-01', 'purchase_price': 400.00},
            {'symbol': 'QQQ', 'shares': 75, 'purchase_date': '2023-02-01', 'purchase_price': 320.00},
            {'symbol': 'TSLA', 'shares': 25, 'purchase_date': '2023-04-01', 'purchase_price': 200.00}
        ]
    }
    return template

def export_to_excel(analyzer, filename="portfolio_risk_report.xlsx"):
    """
    Export risk analysis results to Excel file
    """
    if not analyzer.performance_metrics or not analyzer.risk_metrics:
        print("Please run the analysis first")
        return
    
    with pd.ExcelWriter(filename, engine='openpyxl') as writer:
        # Portfolio summary
        if analyzer.portfolio_data is not None:
            analyzer.portfolio_data.to_excel(writer, sheet_name='Portfolio Holdings', index=False)
        
        # Performance metrics
        perf_df = pd.DataFrame(analyzer.performance_metrics).T
        perf_df.to_excel(writer, sheet_name='Performance Metrics')
        
        # Risk metrics
        risk_df = pd.DataFrame(analyzer.risk_metrics).T
        risk_df.to_excel(writer, sheet_name='Risk Metrics')
        
        # Returns data
        if analyzer.returns_data:
            returns_df = pd.DataFrame(analyzer.returns_data)
            returns_df.to_excel(writer, sheet_name='Returns Data')
    
    print(f"Report exported to {filename}")

def natural_language_query(analyzer, query):
    """
    Natural language query interface for portfolio analytics
    """
    query = query.lower()
    
    if "performance" in query or "return" in query:
        if analyzer.performance_metrics and 'PORTFOLIO' in analyzer.performance_metrics:
            metrics = analyzer.performance_metrics['PORTFOLIO']
            return f"Portfolio Performance: {metrics['Annualized Return']:.2%} annualized return, {metrics['Volatility']:.2%} volatility, {metrics['Sharpe Ratio']:.2f} Sharpe ratio"
    
    elif "risk" in query or "volatility" in query:
        if analyzer.risk_metrics and 'PORTFOLIO' in analyzer.risk_metrics:
            risk = analyzer.risk_metrics['PORTFOLIO']
            return f"Portfolio Risk: {risk['Volatility']:.2%} volatility, {risk['Beta']:.2f} beta, {risk['VaR 95%']:.2%} VaR 95%"
    
    elif "drawdown" in query:
        if analyzer.performance_metrics and 'PORTFOLIO' in analyzer.performance_metrics:
            max_dd = analyzer.performance_metrics['PORTFOLIO']['Max Drawdown']
            return f"Maximum Drawdown: {max_dd:.2%}"
    
    elif "correlation" in query:
        if analyzer.risk_metrics and 'PORTFOLIO' in analyzer.risk_metrics:
            corr = analyzer.risk_metrics['PORTFOLIO']['Correlation with S&P 500']
            return f"Correlation with S&P 500: {corr:.2f}"
    
    else:
        return "I can help you with performance, risk, drawdown, and correlation analysis. Please ask about these topics."

# Example usage of the complete system
print("Portfolio Risk Analysis Platform Ready!")
print("Features:")
print("✓ Portfolio data ingestion from Jupyter notebooks")
print("✓ Performance metrics calculation")
print("✓ Risk metrics and decomposition")
print("✓ Scenario analysis and stress testing")
print("✓ Comprehensive visualizations")
print("✓ Excel export capability")
print("✓ Natural language query interface")
print()
print("To use: Create a PortfolioRiskAnalyzer instance and run the analysis pipeline")


# Portfolio Risk Analysis Platform

## Overview
This notebook contains a comprehensive AI-driven portfolio risk and analytics platform designed for professional investment managers. The platform provides:

### Key Features
- **Portfolio Data Ingestion**: Loads portfolio data from Jupyter notebooks with stocks/ETFs and purchase information
- **Performance Metrics**: Calculates annualized returns, Sharpe ratio, drawdown, capture ratios, and multi-horizon returns
- **Risk Metrics**: Provides volatility, VaR, CVaR, beta, and correlation analysis
- **Scenario Analysis**: Stress testing and scenario analysis including market crashes and volatility spikes
- **Visualizations**: Comprehensive charts for performance, risk-return profiles, drawdowns, and correlations
- **Export Capabilities**: Excel export for explainability and further analysis
- **Natural Language Interface**: Query portfolio analytics using natural language

### Usage Instructions

1. **Load Portfolio Data**: Use `analyzer.load_portfolio_from_notebook("your_notebook.ipynb")` to load your portfolio
2. **Fetch Price Data**: Run `analyzer.fetch_price_data()` to get historical prices
3. **Calculate Returns**: Execute `analyzer.calculate_returns()` to compute returns
4. **Generate Metrics**: Run performance and risk calculations
5. **Create Report**: Generate comprehensive risk report with visualizations

### Expected Portfolio Data Format
```python
portfolio_data = [
    {'symbol': 'AAPL', 'shares': 100, 'purchase_date': '2023-01-15', 'purchase_price': 150.00},
    {'symbol': 'MSFT', 'shares': 50, 'purchase_date': '2023-02-20', 'purchase_price': 250.00},
    # ... more positions
]
```

### Analytics Provided
- **Performance Metrics**: Total return, annualized return, volatility, Sharpe ratio, max drawdown, upside/downside capture
- **Risk Metrics**: VaR, CVaR, beta, correlation with S&P 500
- **Scenario Analysis**: Market crash scenarios, volatility spikes, interest rate shocks
- **Visualizations**: Performance charts, risk-return scatter plots, drawdown analysis, correlation heatmaps
